# Notebook 04 — CRI Computation

## Crisis Probability and Credit Risk Index Construction

This notebook constructs currency-specific and system-wide Credit Risk Indices using OU parameters from Notebook 03.

**Contents:**
1. Crisis Thresholds (90th, 95th, 99th percentiles)
2. Crisis Probability Time Series
3. Currency-Specific CRIs
4. System-Wide Composite CRI
5. Visualizations (Figures 6–8)

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats as sp_stats
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.figsize': (12, 6), 'figure.dpi': 150, 'savefig.dpi': 300,
    'font.size': 11, 'axes.titlesize': 14, 'axes.labelsize': 12,
    'legend.fontsize': 10, 'font.family': 'serif'
})

print('Libraries loaded.')

In [ ]:
# ─── Load Data and Parameters ────────────────────────────────────────────────
usd = pd.read_csv('../data/processed/spreads_usd_new_amount.csv', parse_dates=['date'], index_col='date')
khr = pd.read_csv('../data/processed/spreads_khr_new_amount.csv', parse_dates=['date'], index_col='date')

S_usd = usd['spread'].values
S_khr = khr['spread'].values
dates = usd.index
dt = 1/12

# Load OU parameters from Notebook 03
# If ou_parameters_mle.csv exists, load it; otherwise re-estimate
try:
    params = pd.read_csv('../data/processed/ou_parameters_mle.csv')
    params = params.set_index('parameter')
    kappa_usd = params.loc['kappa', 'USD']
    theta_usd = params.loc['theta', 'USD']
    sigma_usd = params.loc['sigma', 'USD']
    kappa_khr = params.loc['kappa', 'KHR']
    theta_khr = params.loc['theta', 'KHR']
    sigma_khr = params.loc['sigma', 'KHR']
    print('Loaded OU parameters from ou_parameters_mle.csv')
except FileNotFoundError:
    print('WARNING: ou_parameters_mle.csv not found. Please run Notebook 03 first.')
    print('Re-estimating parameters...')
    
    def ou_neg_log_likelihood(params, data, dt):
        kappa, theta, sigma = params
        if kappa <= 0 or sigma <= 0:
            return 1e10
        n = len(data) - 1
        exp_kdt = np.exp(-kappa * dt)
        m = theta + (data[:-1] - theta) * exp_kdt
        v = (sigma**2 / (2 * kappa)) * (1 - np.exp(-2 * kappa * dt))
        if v <= 0:
            return 1e10
        residuals = data[1:] - m
        ll = -0.5 * n * np.log(2 * np.pi) - 0.5 * n * np.log(v) - 0.5 * np.sum(residuals**2) / v
        return -ll
    
    for data, label in [(S_usd, 'USD'), (S_khr, 'KHR')]:
        best_nll = np.inf
        best_x = None
        for k0 in [0.5, 1.0, 2.0, 5.0, 10.0]:
            result = minimize(ou_neg_log_likelihood, [k0, np.mean(data), np.std(np.diff(data))*np.sqrt(12)],
                            args=(data, dt), method='Nelder-Mead', options={'maxiter': 50000})
            if result.fun < best_nll and result.x[0] > 0 and result.x[2] > 0:
                best_nll = result.fun
                best_x = result.x
        if label == 'USD':
            kappa_usd, theta_usd, sigma_usd = best_x
        else:
            kappa_khr, theta_khr, sigma_khr = best_x
    print('Parameters re-estimated.')

print(f'\nUSD: κ={kappa_usd:.4f}, θ={theta_usd:.4f}%, σ={sigma_usd:.4f}')
print(f'KHR: κ={kappa_khr:.4f}, θ={theta_khr:.4f}%, σ={sigma_khr:.4f}')

---
## 1. Crisis Thresholds

In [ ]:
# ─── Crisis Thresholds ───────────────────────────────────────────────────────
thresholds = {}
for pct in [90, 95, 99]:
    thresholds[f'USD_P{pct}'] = np.percentile(S_usd, pct)
    thresholds[f'KHR_P{pct}'] = np.percentile(S_khr, pct)

# Primary threshold: 95th percentile
Sc_usd = thresholds['USD_P95']
Sc_khr = thresholds['KHR_P95']

print('═══════════════════════════════════════════════════')
print('         Crisis Thresholds (Spread Levels)')
print('═══════════════════════════════════════════════════')
print(f'{"Percentile":<15} {"USD Spread (%)":<18} {"KHR Spread (%)":<18}')
print('─' * 51)
for pct in [90, 95, 99]:
    marker = ' ← PRIMARY' if pct == 95 else ''
    print(f'P{pct:<14} {thresholds[f"USD_P{pct}"]:<18.4f} {thresholds[f"KHR_P{pct}"]:<18.4f}{marker}')
print('═══════════════════════════════════════════════════')

---
## 2. Crisis Probability Time Series

$$P(S_t^c > S_c^c) = 1 - \Phi\left(\frac{S_c^c - m^c(t)}{\sqrt{v^c(t)}}\right)$$

where:
- $m^c(t) = \theta^c + (S_{t-1}^c - \theta^c)e^{-\kappa^c \Delta t}$ (conditional mean)
- $v^c(t) = \frac{(\sigma^c)^2}{2\kappa^c}(1 - e^{-2\kappa^c \Delta t})$ (conditional variance)

In [ ]:
# ─── Compute Crisis Probabilities ────────────────────────────────────────────
def compute_crisis_probability(data, kappa, theta, sigma, Sc, dt):
    """
    Compute crisis probability P(S_t > Sc) at each time step.
    Returns array of probabilities (length = len(data)).
    First value uses unconditional distribution.
    """
    n = len(data)
    prob = np.zeros(n)
    
    exp_kdt = np.exp(-kappa * dt)
    v = (sigma**2 / (2 * kappa)) * (1 - np.exp(-2 * kappa * dt))
    
    # First observation: use unconditional distribution
    v_uncond = sigma**2 / (2 * kappa)
    prob[0] = 1 - sp_stats.norm.cdf(Sc, loc=theta, scale=np.sqrt(v_uncond))
    
    # Subsequent observations: use conditional distribution
    for t in range(1, n):
        m_t = theta + (data[t-1] - theta) * exp_kdt
        prob[t] = 1 - sp_stats.norm.cdf(Sc, loc=m_t, scale=np.sqrt(v))
    
    return prob

# Compute for both currencies
P_usd = compute_crisis_probability(S_usd, kappa_usd, theta_usd, sigma_usd, Sc_usd, dt)
P_khr = compute_crisis_probability(S_khr, kappa_khr, theta_khr, sigma_khr, Sc_khr, dt)

print(f'USD crisis probability — mean: {P_usd.mean():.4f}, max: {P_usd.max():.4f}')
print(f'KHR crisis probability — mean: {P_khr.mean():.4f}, max: {P_khr.max():.4f}')

In [ ]:
# ─── FIGURE 6: Crisis Probability Over Time ──────────────────────────────────
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(dates, P_usd, color='#1565C0', linewidth=1.3, label='USD Crisis Probability', alpha=0.9)
ax.plot(dates, P_khr, color='#C62828', linewidth=1.3, label='KHR Crisis Probability', alpha=0.9)

# COVID shading
ax.axvspan(pd.Timestamp('2020-01-01'), pd.Timestamp('2021-12-31'),
           alpha=0.12, color='grey', label='COVID-19 Period')

# Event annotations
events = [
    ('2020-03-01', 'COVID-19', 0.85),
    ('2022-03-01', 'Fed Hikes', 0.85),
    ('2024-09-01', 'Fed Cuts', 0.85),
]
for date, label, ypos in events:
    ax.annotate(label, xy=(pd.Timestamp(date), ypos),
                fontsize=8, ha='center',
                bbox=dict(boxstyle='round,pad=0.2', facecolor='lightyellow',
                          edgecolor='grey', alpha=0.7))

ax.set_xlabel('Date')
ax.set_ylabel('Crisis Probability P(S > S_c)')
ax.set_title('Figure 6: Crisis Probability Over Time (95th Percentile Threshold)',
             fontweight='bold', fontsize=13)
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)
ax.set_ylim(-0.02, 1.02)
ax.xaxis.set_major_locator(mdates.YearLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
plt.xticks(rotation=45)

plt.tight_layout()
plt.savefig('../figures/fig6_crisis_probability.png', dpi=300, bbox_inches='tight')
plt.show()
print('Saved: fig6_crisis_probability.png')

---
## 3. Currency-Specific CRIs

$$\text{CRI}^c_t = 0.5 \cdot \hat{\sigma}^c_{\text{normalized}} + 0.5 \cdot P(S_t^c > S_c^c)$$

We normalize σ to [0, 1] range for comparability by dividing by the maximum possible σ observed.

In [ ]:
# ─── Currency-Specific CRI ───────────────────────────────────────────────────
# Normalize sigma to [0,1] range for CRI combination
sigma_max = max(sigma_usd, sigma_khr) * 1.5  # scale factor for normalization
sigma_usd_norm = sigma_usd / sigma_max
sigma_khr_norm = sigma_khr / sigma_max

# CRI = 0.5 * normalized_sigma + 0.5 * crisis_probability
CRI_usd = 0.5 * sigma_usd_norm + 0.5 * P_usd
CRI_khr = 0.5 * sigma_khr_norm + 0.5 * P_khr

print('Currency-Specific CRI Statistics:')
print(f'  CRI_USD — mean: {CRI_usd.mean():.4f}, max: {CRI_usd.max():.4f}, min: {CRI_usd.min():.4f}')
print(f'  CRI_KHR — mean: {CRI_khr.mean():.4f}, max: {CRI_khr.max():.4f}, min: {CRI_khr.min():.4f}')

In [ ]:
# ─── FIGURE 7: Currency-Specific CRIs ────────────────────────────────────────
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(dates, CRI_usd, color='#1565C0', linewidth=1.5, label='CRI (USD)', alpha=0.9)
ax.plot(dates, CRI_khr, color='#C62828', linewidth=1.5, label='CRI (KHR)', alpha=0.9)

# COVID shading
ax.axvspan(pd.Timestamp('2020-01-01'), pd.Timestamp('2021-12-31'),
           alpha=0.12, color='grey', label='COVID-19 Period')

# Risk level bands
ax.axhline(y=0.3, color='green', linestyle=':', alpha=0.4, linewidth=0.8)
ax.axhline(y=0.6, color='orange', linestyle=':', alpha=0.4, linewidth=0.8)
ax.text(dates[-1] + pd.Timedelta(days=30), 0.15, 'LOW', color='green', fontsize=8, fontweight='bold')
ax.text(dates[-1] + pd.Timedelta(days=30), 0.45, 'MODERATE', color='orange', fontsize=8, fontweight='bold')
ax.text(dates[-1] + pd.Timedelta(days=30), 0.75, 'HIGH', color='red', fontsize=8, fontweight='bold')

ax.set_xlabel('Date')
ax.set_ylabel('Credit Risk Index')
ax.set_title('Figure 7: Currency-Specific Credit Risk Indices (CRI)',
             fontweight='bold', fontsize=13)
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)
ax.set_ylim(-0.02, 1.02)
ax.xaxis.set_major_locator(mdates.YearLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
plt.xticks(rotation=45)

plt.tight_layout()
plt.savefig('../figures/fig7_cri_timeseries.png', dpi=300, bbox_inches='tight')
plt.show()
print('Saved: fig7_cri_timeseries.png')

---
## 4. System-Wide Composite CRI

$$\text{CRI}_{\text{System},t} = w_{USD} \cdot \text{CRI}^{USD}_t + w_{KHR} \cdot \text{CRI}^{KHR}_t$$

Three weighting schemes:
1. **Loan-share**: ~80% USD / 20% KHR (reflects actual market composition)
2. **Equal**: 50% / 50%
3. **KHR-weighted**: 20% USD / 80% KHR (sensitivity check)

In [ ]:
# ─── System-Wide CRI ─────────────────────────────────────────────────────────
# Weighting schemes
weights = {
    'Loan-Share (80/20)': (0.80, 0.20),
    'Equal (50/50)':      (0.50, 0.50),
    'KHR-Heavy (20/80)':  (0.20, 0.80)
}

CRI_system = {}
for name, (w_usd, w_khr) in weights.items():
    CRI_system[name] = w_usd * CRI_usd + w_khr * CRI_khr
    print(f'CRI_System ({name}) — mean: {CRI_system[name].mean():.4f}, max: {CRI_system[name].max():.4f}')

In [ ]:
# ─── FIGURE 8: System CRI ────────────────────────────────────────────────────
fig, ax = plt.subplots(figsize=(14, 7))

# Individual CRIs (faded)
ax.plot(dates, CRI_usd, color='#1565C0', linewidth=0.8, alpha=0.3, linestyle='--', label='CRI (USD)')
ax.plot(dates, CRI_khr, color='#C62828', linewidth=0.8, alpha=0.3, linestyle='--', label='CRI (KHR)')

# System CRIs
colors_sys = ['#4A148C', '#E65100', '#1B5E20']
for (name, cri), color in zip(CRI_system.items(), colors_sys):
    ax.plot(dates, cri, color=color, linewidth=1.8, alpha=0.85, label=f'System CRI — {name}')

# COVID shading
ax.axvspan(pd.Timestamp('2020-01-01'), pd.Timestamp('2021-12-31'),
           alpha=0.1, color='grey', label='COVID-19')

ax.set_xlabel('Date')
ax.set_ylabel('Credit Risk Index')
ax.set_title('Figure 8: System-Wide Composite Credit Risk Index',
             fontweight='bold', fontsize=13)
ax.legend(loc='upper right', fontsize=9)
ax.grid(True, alpha=0.3)
ax.set_ylim(-0.02, 1.02)
ax.xaxis.set_major_locator(mdates.YearLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
plt.xticks(rotation=45)

plt.tight_layout()
plt.savefig('../figures/fig8_system_cri.png', dpi=300, bbox_inches='tight')
plt.show()
print('Saved: fig8_system_cri.png')

In [ ]:
# ─── CRI Summary Table (Table 3) ────────────────────────────────────────────
cri_df = pd.DataFrame({
    'Date': dates,
    'USD_Spread': S_usd,
    'KHR_Spread': S_khr,
    'P_crisis_USD': P_usd,
    'P_crisis_KHR': P_khr,
    'CRI_USD': CRI_usd,
    'CRI_KHR': CRI_khr,
    'CRI_System_LoanShare': CRI_system['Loan-Share (80/20)'],
    'CRI_System_Equal': CRI_system['Equal (50/50)']
})
cri_df.to_csv('../data/processed/cri_results.csv', index=False)

print('\n═══════════════════════════════════════════════════════════')
print('     TABLE 3: CRI Summary Statistics')
print('═══════════════════════════════════════════════════════════')
summary = pd.DataFrame({
    'Metric': ['Mean', 'Max', 'Min', 'Std'],
    'CRI_USD': [CRI_usd.mean(), CRI_usd.max(), CRI_usd.min(), CRI_usd.std()],
    'CRI_KHR': [CRI_khr.mean(), CRI_khr.max(), CRI_khr.min(), CRI_khr.std()],
    'CRI_System': [CRI_system['Loan-Share (80/20)'].mean(), CRI_system['Loan-Share (80/20)'].max(),
                   CRI_system['Loan-Share (80/20)'].min(), CRI_system['Loan-Share (80/20)'].std()]
}).set_index('Metric')
print(summary.round(4).to_string())
print('═══════════════════════════════════════════════════════════')
print('\nSaved: cri_results.csv')

---
## Summary

Key CRI findings:

1. **Crisis thresholds** are currency-specific — the KHR threshold is naturally higher due to the higher mean spread.
2. **Crisis probability** captures time-varying risk that tracks the actual spread dynamics.
3. **System CRI** with loan-share weighting is dominated by USD dynamics (80% of lending), but the KHR component contributes disproportionate volatility.
4. **CRI results saved to `cri_results.csv`** for use in stress testing (NB 05) and COVID analysis (NB 06).